In [ ]:
from utils.const import *
import duckdb
import teehr.queries.duckdb as tqd

In [ ]:
%%time
# print out the query to replace reading from joined parquet.
df = tqd.get_metrics(
    primary_filepath=LOCAL_PRIMARY_FILEPATH,
    secondary_filepath=LOCAL_SECONDARY_FILEPATH,
    crosswalk_filepath=LOCAL_CROSSWALK_FILEPATH,
    include_metrics=["root_mean_squared_error", "relative_bias", "kling_gupta_efficiency", "r_squared"],
    group_by=["primary_location_id", "configuration"],
    order_by=["primary_location_id", "configuration"],
    remove_duplicates=False,
    return_query=True,
)
print(df)

In [ ]:
%%time
df = duckdb.query(f"""
WITH joined AS (
        SELECT * FROM '{LOCAL_JOINED_FILEPATH}'
    )
    , metrics AS (
        SELECT
            joined.primary_location_id,joined.configuration
            , 1 - sqrt(
        pow(corr(secondary_value, primary_value) - 1, 2)
        + pow(stddev(secondary_value)
            / stddev(primary_value) - 1, 2)
        + pow(avg(secondary_value) / avg(primary_value) - 1, 2)
    ) as kling_gupta_efficiency
            , sqrt(sum(power(absolute_difference, 2))/count(*))
        as root_mean_squared_error
            , sum(secondary_value - primary_value) / sum(primary_value) AS relative_bias
            , pow(corr(secondary_value, primary_value), 2) as r_squared
        FROM
            joined
        GROUP BY
            joined.primary_location_id,joined.configuration
    )
    SELECT
        metrics.*
    FROM metrics
    ORDER BY
        metrics.primary_location_id,metrics.configuration
;
""").to_df()
df

In [ ]:
df.to_parquet("/data/benchmarks/teehr-benchmark-202404/results/validation_teehr_metric_values.parquet")